In [1]:
import os
import netket as nk
from netket.operator.spin import sigmax, sigmaz, sigmay
# from src.wilson import * 
from src.models import * 
import matplotlib.pyplot as plt

In [2]:
L = 8
g = nk.graph.Square(L, pbc=True)
print(f'g.n_edges = {g.n_edges}, LxLx2 = {L**2 * 2}')
hi = nk.hilbert.Spin(s=1/2, N=g.n_edges + g.n_nodes)
theta = np.pi / 4
phi = np.pi / 4

g.n_edges = 128, LxLx2 = 128


The $\mathbb{Z}_2$ gauge theory with matter Hamiltonian is 
$$
H = -\cos\theta \sum_{\square}\sum_{\langle ij\rangle \in\square} \sigma_{ij}^z - \sin\theta \sum_{\langle ij \rangle} \sigma_{ij}^x
- \cos \phi \sum_{\langle ij\rangle} \tau_i^z \sigma_{ij}^z \tau_j^z - \sin\phi \sum_i \tau^x_i
$$

In [3]:
def z2_matter_hamiltonian(theta, phi, g, hi):
	h = 0
	edges = ordered_edges(g)

	for i in g.nodes(): 
		l, r, u, d = plaquette_indices(i, L)
		h += - np.cos(theta) * sigmaz(hi, l) * sigmaz(hi, d) * sigmaz(hi, r) * sigmaz(hi, u)

		h += - np.sin(phi) * sigmaz(hi, 2*L**2 + i)

	i = 0
	for e in edges:
		h += - np.sin(theta) * sigmax(hi, i)
		h += - np.cos(theta) * sigmaz(hi, 2*L**2 + e[0]) * sigmaz(hi, i) * sigmaz(hi, 2*L**2 + e[1])
		i += 1
	return h

# z2_matter_hamiltonian(np.pi/4, np.pi/4, g, hi)

In [4]:
gerbil = GERBIL(graph=g)
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=gerbil, n_samples=1008)
optimizer = nk.optimizer.Sgd(learning_rate=0.05)
preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
h = z2_matter_hamiltonian(theta, phi, g, hi)
vmc = nk.driver.VMC(h, optimizer, variational_state=vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
vmc.run(n_iter=10, out=log)

100%|██████████| 10/10 [00:24<00:00,  2.46s/it, Energy=-129.10+0.02j ± 0.32 [σ²=100.10, R̂=1.0136]]


(RuntimeLog():
  keys = ['Energy'],)